# Imports

In [1]:
import re
import collections


import numpy as np
import pandas as pd
from tokenizers import CharBPETokenizer, Tokenizer
from scipy.sparse import lil_matrix


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Methods

In [2]:
def get_vocab(filename):
    vocab = collections.defaultdict(int)
    with open(filename, 'r', encoding='utf-8') as fhand:
        for line in fhand:
            words = line.lower().strip().split()
            for word in words:
                vocab[' '.join(list(word))] += 1
    return vocab


def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs


def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out


def get_tokens(vocab):
    tokens = collections.defaultdict(int)
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens[token] += freq
    return tokens


def get_tokens_from_vocab(vocab):
    tokens_frequencies = collections.defaultdict(int)
    vocab_tokenization = {}
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens_frequencies[token] += freq
        vocab_tokenization[''.join(word_tokens)] = word_tokens
    return tokens_frequencies, vocab_tokenization


def measure_token_length(token):
    
    return len(token)


def tokenize_text(string, sorted_tokens, unknown_token='</u>'):
    
    if string == '':
        return []
    if sorted_tokens == []:
        return [unknown_token]

    string_tokens = []
    for i in range(len(sorted_tokens)):
        token = sorted_tokens[i]
        token_reg = re.escape(token.replace('.', '[.]'))

        matched_positions = [(m.start(0), m.end(0)) for m in re.finditer(token_reg, string)]
        if len(matched_positions) == 0:
            continue
        substring_end_positions = [matched_position[0] for matched_position in matched_positions]

        substring_start_position = 0
        for substring_end_position in substring_end_positions:
            substring = string[substring_start_position:substring_end_position]
            string_tokens += tokenize_text(string=substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
            string_tokens += [token]
            substring_start_position = substring_end_position + len(token)
        remaining_substring = string[substring_start_position:]
        string_tokens += tokenize_text(string=remaining_substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
        break
    return string_tokens

# Ex. 1.1

In [8]:
# load vocab
vocab = get_vocab('./data/lenta.txt')

In [9]:
print('==========')
print('Tokens Before BPE')
tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
print('All tokens: {}'.format(tokens_frequencies.keys()))
print('Number of tokens: {}'.format(len(tokens_frequencies.keys())))
print('==========')

Tokens Before BPE
All tokens: dict_keys(['б', 'о', 'и', 'у', 'с', 'п', 'ц', 'к', 'н', 'а', 'д', 'р', 'е', 'з', 'ч', 'л', 'ь', 'т', 'м', 'г', 'в', '.', 'я', ',', 'ш', 'й', 'ю', 'ж', 'ы', '1', '4', 'х', 'щ', '«', '»', '6', '9', '(', '2', '-', ')', 'э', 'ф', '—', 'd', 'a', 's', 'i', 't', 'n', 'e', 'r', 'o', 'f', '!', 'ъ', '7', '5', '3', '№', '…', '–', 'l', 'y', 'm', '8', '0', '"', ':', 'g', 'z', 'u', '$', '%', 'b', 'k', 'w', 'x', 'p', 'h', 'v', "'", 'j', 'ё', '/', 'c', ';', '=', '&', '?', 'q', '*', '@', '+', '’', '·', '“', '”', '•', '>', '_', '|', '[', ']', '\xad', '~', '<', '#', '£', 'ї', 'і', '`'])
Number of tokens: 112


In [10]:
num_merges = 20
k = 5
for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best_k = sorted(pairs, key=pairs.get, reverse=True)[:k]
    print(best_k)
    for best in best_k:
        vocab = merge_vocab(best, vocab)
    print('Iter: {}'.format(i))
    tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
    print('Tokens: {}'.format(tokens_frequencies))
    print('Number of tokens: {}'.format(len(tokens_frequencies)))
    print('==========')


[('с', 'т'), ('е', 'н'), ('н', 'и'), ('о', 'в'), ('н', 'о')]
Iter: 0
Tokens: defaultdict(<class 'int'>, {'б': 157356, 'о': 820024, 'и': 680396, 'у': 216731, 'с': 396303, 'п': 289248, 'ц': 61070, 'к': 311128, 'н': 358093, 'а': 755381, 'д': 290446, 'р': 541120, 'ен': 128279, 'з': 146086, 'ч': 119236, 'л': 380097, 'ь': 120493, 'т': 433175, 'ст': 150095, 'е': 678575, 'м': 282793, 'г': 157465, 'в': 348663, '.': 90043, 'я': 165587, ',': 123414, 'ш': 54185, 'ов': 119049, 'й': 119710, 'ю': 55438, 'ни': 75181, 'ж': 76178, 'ы': 165244, '1': 15992, '4': 4949, 'но': 85526, 'х': 84224, 'щ': 39272, '«': 52, '»': 52, '6': 4443, '9': 9227, '(': 4249, '2': 11808, '-': 32382, ')': 4274, 'э': 27050, 'ф': 37385, '—': 57, 'd': 3742, 'a': 9930, 's': 9440, 'i': 8499, 't': 8438, 'n': 8640, 'e': 13183, 'r': 9371, 'o': 9635, 'f': 2030, '!': 157, 'ъ': 4732, '7': 4085, '5': 7024, '3': 6146, '№': 100, '…': 6, '–': 48, 'l': 4882, 'y': 1678, 'm': 4400, '8': 4121, '0': 18502, '"': 60607, ':': 3781, 'g': 1932, 'z': 31

[('к', 'и'), ('е', 'д'), ('в', 'а'), ('л', 'е'), ('д', 'е')]
Iter: 6
Tokens: defaultdict(<class 'int'>, {'б': 110367, 'о': 347202, 'и': 485159, 'у': 216731, 'с': 396303, 'по': 94674, 'ц': 61070, 'ки': 37980, 'на': 98362, 'д': 176776, 'р': 167982, 'к': 130973, 'ени': 44670, 'за': 49998, 'ко': 87718, 'н': 155130, 'ч': 119236, 'ли': 67533, 'ь': 64682, 'т': 203179, 'ст': 150095, 'п': 106953, 'л': 178187, 'е': 300094, 'м': 237739, 'г': 85897, 'ер': 63230, 'а': 264451, 'в': 266961, '.': 90043, 'не': 49016, 'пр': 87621, 'я': 165587, 'те': 44525, 'ль': 55811, ',': 123414, 'з': 96088, 'ш': 54185, 'ра': 109367, 'ов': 119049, 'й': 119710, 'ю': 55438, 'ре': 61690, 'ни': 75181, 'ж': 76178, 'ы': 109659, 'ка': 54457, 'го': 71568, '1': 15992, '4': 4949, 'ен': 83609, 'но': 85526, 'х': 84224, 'об': 46989, 'де': 29707, 'ва': 36493, 'ла': 48554, 'та': 46582, 'ро': 51230, 'то': 75434, 'ны': 55585, 'ми': 45054, 'щ': 39272, 'ет': 63455, 'ле': 30012, 'ед': 36846, '«': 52, '»': 52, '6': 4443, '9': 9227, 'да': 

[('ст', 'и'), ('в', 'ы'), ('с', 'е'), ('и', 'з'), ('п', 'а')]
Iter: 11
Tokens: defaultdict(<class 'int'>, {'б': 110367, 'о': 169379, 'и': 263430, 'у': 186536, 'со': 34494, 'по': 94674, 'ц': 27403, 'ки': 37980, 'на': 98362, 'д': 123695, 'ру': 30195, 'с': 255635, 'к': 130973, 'ени': 44670, 'за': 49998, 'ко': 63357, 'н': 155130, 'ч': 93325, 'ли': 67533, 'ь': 37186, 'от': 27863, 'ст': 101134, 'п': 85508, 'л': 144898, 'е': 252139, 'м': 181024, 'г': 85897, 'ер': 63230, 'ма': 28902, 'в': 210253, '.': 90043, 'не': 49016, 'пр': 60566, 'я': 134897, 'те': 44525, 'ль': 55811, ',': 123414, 'з': 74506, 'ш': 54185, 'се': 22044, 'ра': 109367, 'ов': 92361, 'а': 160746, 'р': 111141, 'ти': 28191, 'й': 119710, 'ю': 55438, 'ре': 61690, 'ско': 24361, 'ни': 75181, 'т': 119629, 'сти': 22291, 'ж': 76178, 'ы': 87452, 'ка': 54457, 'го': 71568, '1': 15992, '4': 4949, 'ен': 83609, 'до': 27309, 'но': 85526, 'х': 84224, 'об': 46989, 'ть': 27496, 'ся': 30690, 'ци': 33667, 'де': 29707, 'ло': 33289, 'па': 21445, 'ва': 

[('пр', 'и'), ('г', 'а'), ('ени', 'я'), ('р', 'ы'), ('но', 'й')]
Iter: 15
Tokens: defaultdict(<class 'int'>, {'бо': 20697, 'и': 226889, 'у': 186536, 'со': 34494, 'по': 94674, 'ц': 27403, 'ки': 21360, 'на': 98362, 'д': 123695, 'ру': 30195, 'с': 204972, 'к': 130973, 'ени': 28693, 'за': 49998, 'ко': 63357, 'н': 122774, 'ч': 54758, 'ли': 67533, 'ь': 37186, 'от': 27863, 'ст': 101134, 'п': 68950, 'л': 109532, 'е': 215744, 'м': 161187, 'г': 69828, 'ер': 63230, 'ма': 28902, 'в': 210253, '.': 90043, 'не': 49016, 'при': 16124, 'я': 99994, 'тель': 16649, ',': 123414, 'б': 72302, 'з': 74506, 'ш': 54185, 'се': 22044, 'ра': 109367, 'о': 130529, 'ов': 62269, 'ча': 19444, 'а': 107630, 'р': 95792, 'ти': 28191, 'й': 104546, 'ю': 55438, 'ре': 61690, 'ско': 24361, 'ни': 75181, 'т': 119629, 'сти': 22291, 'ж': 76178, 'ы': 54735, 'ка': 54457, 'ры': 15349, 'го': 71568, '1': 15992, '4': 4949, 'ен': 83609, 'до': 27309, 'но': 70362, 'пр': 44442, 'ения': 15977, 'пе': 16558, 'х': 67090, 'об': 46989, 'ть': 27496, '

[('ци', 'и'), ('х', 'о'), ('ен', 'т'), ('ро', 'сси'), ('п', 'ра')]
Iter: 19
Tokens: defaultdict(<class 'int'>, {'бо': 20697, 'и': 186079, 'у': 129057, 'со': 34494, 'по': 94674, 'ц': 27403, 'ки': 21360, 'на': 98362, 'д': 108635, 'ру': 30195, 'с': 175430, 'к': 116688, 'ени': 15318, 'за': 49998, 'ко': 63357, 'н': 109546, 'чи': 14989, 'ли': 67533, 'ь': 37186, 'от': 27863, 'ст': 72876, 'п': 56985, 'л': 109532, 'ение': 13375, 'м': 161187, 'г': 69828, 'ер': 63230, 'ма': 28902, 'е': 173867, 'в': 196301, '.': 90043, 'не': 49016, 'при': 16124, 'я': 99994, 'тель': 16649, ',': 123414, 'б': 72302, 'з': 74506, 'ш': 54185, 'се': 22044, 'ра': 97402, 'о': 104143, 'ов': 62269, 'ча': 19444, 'а': 107630, 'р': 95792, 'ти': 28191, 'й': 104546, 'ку': 14285, 'ю': 55438, 'ре': 61690, 'ско': 24361, 'ни': 75181, 'т': 107555, 'сти': 22291, 'же': 14296, 'ы': 54735, 'ка': 54457, 'ры': 15349, 'го': 57756, '1': 15992, '4': 4949, 'ент': 12074, 'до': 27309, 'ного': 13812, 'пр': 15470, 'ж': 61882, 'ения': 15977, 'пе': 1

In [11]:
sorted_tokens_tuple = sorted(tokens_frequencies.items(), key=lambda item: (measure_token_length(item[0]), item[1]), reverse=True)
sorted_tokens = [token for (token, freq) in sorted_tokens_tuple]

In [12]:
print(sorted_tokens)

['росси', 'тель', 'ения', 'пред', 'ного', 'ение', 'про', 'ова', 'ста', 'ско', 'сти', 'что', 'ово', 'ных', 'ски', 'при', 'ени', 'ной', 'сто', 'пре', 'ств', 'ции', 'ент', 'пра', 'нов', 'общ', 'сси', 'на', 'ра', 'по', 'ни', 'ст', 'ен', 'ли', 'ет', 'ко', 'ер', 'ов', 'ре', 'го', 'но', 'то', 'ка', 'за', 'не', 'ла', 'да', 'та', 'во', 'ми', 'ро', 'ль', 'об', 'ны', 'ва', 'ви', 'со', 'ло', 'ся', 'ру', 'ле', 'де', 'ма', 'ти', 'те', 'от', 'мо', 'ть', 'до', 'ри', 'че', 'ди', 'вы', 'ед', 'се', 'из', 'па', 'ки', 'ци', 'бо', 'ин', 'ме', 'ча', 'ля', 'са', 'бы', 'пе', 'сл', 'га', 'пр', 'ры', 'ду', 'чи', 'су', 'си', 'же', 'ку', 'щи', 'ну', 'хо', 'в', 'и', 'с', 'е', 'м', 'у', ',', 'к', 'н', 'л', 'д', 'а', 'т', 'й', 'о', 'я', 'р', '.', 'з', 'б', 'г', 'ж', '"', 'п', 'ю', 'х', 'ы', 'ш', 'ч', 'ф', 'ь', '-', 'ц', 'э', '0', 'щ', '1', 'e', '2', 'a', 'o', 's', 'r', '9', 'n', 'i', 't', '5', '3', 'c', '4', 'l', 'ъ', '6', 'm', ')', '(', '8', '7', ':', 'd', 'u', 'p', 'b', 'w', 'f', 'h', 'g', 'k', 'y', 'v', 'x', '%', 

In [13]:
yandex_news = '''
Власти Москвы открывают запись на вакцинацию от коронавируса для жителей старше 60 лет с понедельника, 28 декабря. Об этом в своём блоге сообщил мэр столицы Сергей Собянин. Он напомнил, что Минздрав одобрил возможность прививаться от COVID-19 вакциной «Спутник V» гражданам старшего поколения.
'''

In [14]:
print(tokenize_text(string=yandex_news.lower(), sorted_tokens=sorted_tokens, unknown_token='</u>'))

['в', 'ла', 'сти', 'мо', 'с', 'к', 'вы', 'от', 'к', 'ры', 'ва', 'ю', 'т', 'за', 'п', 'и', 'с', 'ь', 'на', 'ва', 'к', 'ци', 'на', 'ци', 'ю', 'от', 'ко', 'ро', 'на', 'ви', 'ру', 'са', 'д', 'ля', 'ж', 'и', 'те', 'ле', 'й', 'ста', 'р', 'ш', 'е', '6', '0', 'л', 'ет', 'с', 'по', 'не', 'де', 'ль', 'ни', 'ка', ',', '2', '8', 'де', 'ка', 'б', 'р', 'я', 'об', 'э', 'то', 'м', 'в', 'с', 'во', 'ё', 'м', 'б', 'ло', 'г', 'е', 'со', 'общ', 'и', 'л', 'м', 'э', 'р', 'сто', 'ли', 'ц', 'ы', 'с', 'ер', 'г', 'е', 'й', 'с', 'об', 'я', 'ни', 'н', 'о', 'н', 'на', 'по', 'м', 'ни', 'л', ',', 'что', 'ми', 'н', 'з', 'д', 'ра', 'в', 'о', 'д', 'об', 'ри', 'л', 'во', 'з', 'мо', 'ж', 'но', 'ст', 'ь', 'при', 'ви', 'ва', 'ть', 'ся', 'от', 'c', 'o', 'v', 'i', 'd', '-', '1', '9', 'ва', 'к', 'ци', 'ной', '«', 'с', 'п', 'у', 'т', 'ни', 'к', 'v', '»', 'г', 'ра', 'ж', 'да', 'на', 'м', 'ста', 'р', 'ш', 'е', 'го', 'по', 'ко', 'л', 'ения']


В целом по токенам становится видно, что осмысленные части выделяютса, однако цельных слов выделяется все же недостаточно

# Ex 1.2

In [15]:
# load vocab
vocab = get_vocab('./data/lenta.txt')

In [16]:
print('==========')
print('Tokens Before BPE')
tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
print('All tokens: {}'.format(tokens_frequencies.keys()))
print('Number of tokens: {}'.format(len(tokens_frequencies.keys())))
print('==========')

Tokens Before BPE
All tokens: dict_keys(['б', 'о', 'и', 'у', 'с', 'п', 'ц', 'к', 'н', 'а', 'д', 'р', 'е', 'з', 'ч', 'л', 'ь', 'т', 'м', 'г', 'в', '.', 'я', ',', 'ш', 'й', 'ю', 'ж', 'ы', '1', '4', 'х', 'щ', '«', '»', '6', '9', '(', '2', '-', ')', 'э', 'ф', '—', 'd', 'a', 's', 'i', 't', 'n', 'e', 'r', 'o', 'f', '!', 'ъ', '7', '5', '3', '№', '…', '–', 'l', 'y', 'm', '8', '0', '"', ':', 'g', 'z', 'u', '$', '%', 'b', 'k', 'w', 'x', 'p', 'h', 'v', "'", 'j', 'ё', '/', 'c', ';', '=', '&', '?', 'q', '*', '@', '+', '’', '·', '“', '”', '•', '>', '_', '|', '[', ']', '\xad', '~', '<', '#', '£', 'ї', 'і', '`'])
Number of tokens: 112


In [17]:
num_merges = 50
k = 7
for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best_k = sorted(pairs, key=pairs.get, reverse=True)[:k]
    print(best_k)
    for best in best_k:
        vocab = merge_vocab(best, vocab)
    print('Iter: {}'.format(i))
    tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
    print('Tokens: {}'.format(tokens_frequencies))
    print('Number of tokens: {}'.format(len(tokens_frequencies)))
    print('==========')


[('с', 'т'), ('е', 'н'), ('н', 'и'), ('о', 'в'), ('н', 'о'), ('р', 'а'), ('р', 'о')]
Iter: 0
Tokens: defaultdict(<class 'int'>, {'б': 157356, 'о': 735305, 'и': 680396, 'у': 216731, 'с': 396303, 'п': 289248, 'ц': 61070, 'к': 311128, 'н': 358093, 'а': 646014, 'д': 290446, 'р': 347034, 'ен': 128279, 'з': 146086, 'ч': 119236, 'л': 380097, 'ь': 120493, 'т': 433175, 'ст': 150095, 'е': 678575, 'м': 282793, 'г': 157465, 'в': 348663, '.': 90043, 'я': 165587, ',': 123414, 'ш': 54185, 'ра': 109367, 'ов': 119049, 'й': 119710, 'ю': 55438, 'ни': 75181, 'ж': 76178, 'ы': 165244, '1': 15992, '4': 4949, 'но': 85526, 'х': 84224, 'ро': 84719, 'щ': 39272, '«': 52, '»': 52, '6': 4443, '9': 9227, '(': 4249, '2': 11808, '-': 32382, ')': 4274, 'э': 27050, 'ф': 37385, '—': 57, 'd': 3742, 'a': 9930, 's': 9440, 'i': 8499, 't': 8438, 'n': 8640, 'e': 13183, 'r': 9371, 'o': 9635, 'f': 2030, '!': 157, 'ъ': 4732, '7': 4085, '5': 7024, '3': 6146, '№': 100, '…': 6, '–': 48, 'l': 4882, 'y': 1678, 'm': 4400, '8': 4121, '0

[('т', 'и'), ('л', 'о'), ('п', 'ро'), ('с', 'я'), ('р', 'у'), ('д', 'и'), ('п', 'ре')]
Iter: 5
Tokens: defaultdict(<class 'int'>, {'б': 113202, 'о': 249785, 'и': 308760, 'у': 186377, 'со': 34494, 'по': 94674, 'ц': 27403, 'ки': 37980, 'на': 98362, 'д': 213364, 'ру': 30354, 'с': 331119, 'к': 130973, 'ени': 44670, 'за': 49998, 'ко': 87718, 'н': 155509, 'ч': 119236, 'ли': 67533, 'ь': 86482, 'т': 217448, 'ст': 150095, 'п': 134055, 'л': 150951, 'е': 363500, 'м': 237739, 'г': 85897, 'ер': 60329, 'а': 267954, 'в': 230921, '.': 90043, 'не': 48637, 'ри': 47346, 'я': 134897, 'ел': 51861, ',': 123414, 'з': 96088, 'ш': 54185, 'ра': 109367, 'ов': 119049, 'р': 117508, 'ти': 30920, 'й': 119710, 'ю': 55438, 'ре': 61690, 'ни': 75181, 'ж': 76178, 'ы': 109659, 'ка': 54457, 'го': 71568, '1': 15992, '4': 4949, 'ен': 83609, 'но': 85526, 'х': 84224, 'про': 30712, 'ся': 30690, 'ци': 33667, 'ль': 34011, 'ло': 30730, 'об': 44154, 'ва': 36493, 'ла': 45011, 'та': 46622, 'ро': 54007, 'то': 75434, 'ны': 55585, 'ми':

[('ч', 'а'), ('с', 'е'), ('ч', 'то'), ('ов', 'о'), ('т', 'е'), ('н', 'ов'), ('в', 'е')]
Iter: 9
Tokens: defaultdict(<class 'int'>, {'бо': 24849, 'и': 237681, 'у': 186377, 'со': 34494, 'по': 94674, 'ц': 27403, 'ки': 37980, 'на': 98362, 'д': 160085, 'ру': 30354, 'с': 234720, 'к': 130973, 'ени': 44670, 'за': 49998, 'ко': 63843, 'н': 143301, 'ч': 57460, 'ли': 67533, 'ь': 58965, 'от': 21454, 'ст': 101134, 'п': 91911, 'л': 123168, 'е': 212089, 'м': 158230, 'г': 85897, 'ер': 60329, 'ма': 28902, 'в': 190792, '.': 90043, 'не': 48637, 'при': 20699, 'я': 134897, 'тел': 25989, ',': 123414, 'б': 88353, 'з': 76379, 'ш': 54185, 'се': 19273, 'ве': 17922, 'ра': 109367, 'о': 126637, 'ов': 62001, 'ча': 19444, 'а': 144805, 'р': 117508, 'ти': 30920, 'й': 119710, 'ю': 55438, 'ре': 61690, 'ско': 23875, 'ни': 75181, 'т': 124457, 'сти': 22291, 'ж': 76178, 'ел': 25872, 'ы': 87452, 'ка': 54457, 'го': 71568, '1': 15992, '4': 4949, 'ен': 83609, 'до': 29173, 'но': 85526, 'х': 84224, 'про': 30712, 'ть': 27517, 'ся':

[('щ', 'и'), ('ж', 'е'), ('об', 'щ'), ('р', 'ов'), ('пре', 'д'), ('ци', 'и'), ('ст', 'в')]
Iter: 13
Tokens: defaultdict(<class 'int'>, {'бо': 24849, 'и': 181126, 'у': 126308, 'со': 34494, 'по': 94674, 'ц': 27403, 'ки': 21643, 'на': 98362, 'д': 129725, 'ру': 30354, 'с': 172749, 'к': 116688, 'ени': 15312, 'за': 49998, 'ко': 63843, 'н': 114936, 'чи': 15723, 'ли': 67533, 'ь': 42316, 'от': 21454, 'ст': 74582, 'п': 75541, 'л': 108108, 'ение': 13381, 'м': 158230, 'г': 69828, 'ер': 60329, 'ма': 28902, 'е': 169153, 'в': 178535, '.': 90043, 'не': 48637, 'при': 20699, 'я': 103860, 'тель': 16649, ',': 123414, 'б': 70973, 'з': 76379, 'ш': 54185, 'се': 19273, 'ве': 17922, 'ра': 109367, 'о': 112342, 'ов': 49287, 'ча': 19444, 'а': 112464, 'р': 89358, 'ти': 30920, 'й': 104546, 'ку': 14285, 'ю': 55438, 'ре': 61690, 'ско': 23875, 'ни': 75181, 'т': 124457, 'сти': 22291, 'ж': 62993, 'ел': 25872, 'ы': 54636, 'ка': 54457, 'ры': 15436, 'го': 57756, '1': 15992, '4': 4949, 'ен': 83609, 'до': 29173, 'ного': 1381

[('о', 'н'), ('с', 'к'), ('к', 'ов'), ('п', 'у'), ('т', 'у'), ('т', 'ы'), ('б', 'а')]
Iter: 16
Tokens: defaultdict(<class 'int'>, {'бо': 24849, 'и': 146151, 'у': 71061, 'со': 34494, 'по': 83213, 'ц': 27403, 'ки': 21643, 'на': 98362, 'д': 118264, 'ру': 30354, 'ск': 10653, 'ени': 15312, 'к': 97440, 'за': 49998, 'ко': 63843, 'н': 104235, 'чи': 15723, 'ли': 67533, 'сь': 11577, 'от': 21454, 'ст': 74582, 'п': 53179, 'л': 96231, 'ение': 13381, 'м': 146627, 'г': 58225, 'ер': 60329, 'ма': 28902, 'е': 157274, 'в': 178535, '.': 90043, 'не': 48637, 'при': 20699, 'я': 103860, 'тель': 16649, ',': 123414, 'б': 49764, 'зи': 11737, 'ши': 11635, 'с': 150519, 'се': 19273, 'ве': 17922, 'ра': 97402, 'о': 89545, 'ов': 40692, 'ча': 19444, 'а': 102258, 'р': 89358, 'ти': 30920, 'й': 92667, 'ку': 14285, 'ю': 55438, 'ь': 30739, 'бу': 11003, 'ре': 61690, 'ско': 23875, 'ни': 75181, 'т': 91660, 'сти': 22291, 'ж': 62993, 'ел': 25872, 'ы': 44280, 'ка': 54457, 'ры': 15436, 'го': 57756, '1': 15992, '4': 4949, 'ент': 12

...

[('ново', 'сти'), ('з', 'ы'), ('"', 'ново'), ('т', 'ре'), ('ро', 'м'), ('т', 'ро'), ('ст', 'е')]
Iter: 40
Tokens: defaultdict(<class 'int'>, {'бо': 20268, 'и': 94142, 'у': 59321, 'со': 23902, 'по': 68107, 'ц': 19058, 'ки': 21643, 'на': 85211, 'д': 72845, 'ру': 26169, 'ск': 5775, 'ени': 5800, 'к': 58016, 'за': 41068, 'кон': 8961, 'чи': 15723, 'лись': 3713, 'от': 21454, 'сту': 5691, 'п': 32504, 'л': 48721, 'ение': 13381, 'м': 72090, 'г': 29606, 'ер': 22341, 'ма': 28902, 'н': 63155, 'ев': 7254, '.': 80687, 'не': 48637, 'при': 20699, 'я': 53760, 'тель': 16649, ',': 101111, 'бли': 4673, 'зи': 7148, 'вши': 4737, 'сь': 7864, 'с': 93915, 'се': 13592, 'ве': 17922, 'ра': 51775, 'о': 68100, 'сов': 6626, 'ча': 19444, 'ар': 4071, 'ти': 26576, 'й': 63951, 'ку': 14285, 'ю': 31944, 'р': 59610, 'ь': 23943, 'бу': 7244, 'ре': 37052, 'сть': 2982, 'в': 119667, 'ско': 8860, 'ни': 46217, 'ют': 8789, 'сти': 12517, 'е': 74871, 'т': 56144, 'ж': 33646, 'ел': 16263, 'ы': 28687, 'ка': 32638, 'ли': 55418, 'б': 2841

[('сл', 'ова'), ('фе', 'де'), ('ор', 'га'), ('ну', 'ю'), ('ь', 'ю'), ('ба', 'н'), ('ро', 'д')]
Iter: 42
Tokens: defaultdict(<class 'int'>, {'бо': 20268, 'и': 94142, 'у': 59321, 'со': 23902, 'по': 68107, 'ц': 19058, 'ки': 21643, 'на': 85211, 'д': 69500, 'ру': 26169, 'ск': 5775, 'ени': 5800, 'к': 54599, 'за': 41068, 'кон': 8961, 'чи': 15723, 'лись': 3713, 'от': 21454, 'сту': 5691, 'п': 32504, 'л': 48721, 'ение': 13381, 'м': 72090, 'г': 29606, 'ер': 22341, 'ма': 28902, 'н': 59796, 'ев': 7254, '.': 80687, 'не': 48637, 'при': 20699, 'я': 53760, 'тель': 16649, ',': 101111, 'бли': 4673, 'зи': 7148, 'вши': 4737, 'сь': 7864, 'с': 93915, 'се': 13592, 'ве': 17922, 'ра': 51775, 'о': 68100, 'сов': 6626, 'ча': 19444, 'ар': 4071, 'ти': 26576, 'й': 60473, 'ку': 14285, 'ю': 25220, 'р': 59610, 'ь': 20584, 'бу': 7244, 'ре': 37052, 'сть': 2982, 'в': 119667, 'ско': 8860, 'ни': 42740, 'ют': 8789, 'сти': 12517, 'е': 74871, 'т': 56144, 'ж': 33646, 'ел': 16263, 'ы': 28687, 'ка': 29161, 'ли': 55418, 'б': 28410,

[('это', 'м'), ('и', 'х'), ('ч', 'ел'), ('то', 'м'), ('в', 'з'), ('ру', 'к'), ('че', 'ч')]
Iter: 44
Tokens: defaultdict(<class 'int'>, {'бо': 20268, 'и': 90969, 'у': 59321, 'со': 23902, 'по': 68107, 'ц': 19058, 'ки': 21643, 'на': 85211, 'д': 69500, 'ру': 23062, 'ск': 5775, 'ени': 5800, 'к': 51492, 'за': 41068, 'кон': 8961, 'чи': 15723, 'лись': 3713, 'от': 21454, 'сту': 5691, 'п': 29261, 'л': 48721, 'ение': 13381, 'м': 65783, 'г': 26329, 'ер': 22341, 'ма': 28902, 'н': 56487, 'ев': 7254, '.': 80687, 'не': 48637, 'при': 20699, 'я': 53760, 'тель': 16649, ',': 101111, 'бли': 4673, 'зи': 7148, 'вши': 4737, 'сь': 7864, 'с': 93915, 'се': 13592, 'ве': 17922, 'ра': 51775, 'о': 68100, 'сов': 6626, 'ча': 19444, 'ар': 4071, 'ти': 26576, 'й': 60473, 'ку': 14285, 'ю': 25220, 'р': 59610, 'ь': 20584, 'бу': 7244, 'ре': 33734, 'сть': 2982, 'в': 116559, 'ско': 8860, 'ни': 42740, 'ют': 8789, 'сти': 12517, 'е': 74871, 'т': 56144, 'ж': 33646, 'ел': 13096, 'ы': 28687, 'ка': 29161, 'ли': 55418, 'б': 28410, 'ры

[('па', 'р'), ('об', 'ъ'), ('л', 'ения'), ('ме', 'ж'), ('мини', 'ст'), ('то', 'р'), ('ль', 'ко')]
Iter: 46
Tokens: defaultdict(<class 'int'>, {'бо': 20268, 'и': 90969, 'у': 59321, 'со': 23902, 'по': 61947, 'ц': 19058, 'ки': 21643, 'на': 85211, 'д': 69500, 'ру': 23062, 'ск': 5775, 'ени': 5800, 'к': 51492, 'за': 41068, 'кон': 8961, 'чи': 15723, 'лись': 3713, 'от': 21454, 'сту': 5691, 'п': 29261, 'л': 42625, 'ение': 13381, 'м': 65783, 'г': 23284, 'ер': 19249, 'ма': 28902, 'н': 56487, 'ев': 7254, '.': 80687, 'не': 48637, 'при': 20699, 'я': 50664, 'тель': 16649, ',': 101111, 'бли': 4673, 'зи': 7148, 'вши': 4737, 'сь': 7864, 'с': 90823, 'се': 13592, 'ве': 17922, 'ра': 51775, 'о': 68100, 'сов': 6626, 'ча': 19444, 'ар': 4071, 'ти': 26576, 'й': 60473, 'ку': 14285, 'ю': 25220, 'р': 53552, 'ь': 20584, 'бу': 7244, 'ре': 30676, 'сть': 2982, 'в': 116559, 'ско': 8860, 'ни': 42740, 'ют': 8789, 'сти': 12517, 'е': 71826, 'т': 56144, 'ж': 30621, 'ел': 13096, 'ы': 28687, 'ка': 29161, 'ли': 52350, 'б': 284

[('т', 'ру'), ('д', 'ова'), ('чел', 'ов'), ('у', 'ча'), ('ло', 'сь'), ('д', 'ент'), ('д', 'ов')]
Iter: 48
Tokens: defaultdict(<class 'int'>, {'бо': 20268, 'и': 90969, 'у': 56403, 'со': 23902, 'по': 61947, 'ц': 19058, 'ки': 21643, 'на': 82211, 'д': 60741, 'ру': 20111, 'ск': 5775, 'ени': 5800, 'к': 51492, 'за': 41068, 'кон': 8961, 'чи': 15723, 'лись': 3713, 'от': 21454, 'сту': 5691, 'п': 29261, 'л': 42625, 'ение': 13381, 'м': 65783, 'г': 23284, 'ер': 19249, 'ма': 28902, 'н': 56487, 'ев': 7254, '.': 80687, 'не': 48637, 'при': 20699, 'я': 47663, 'тель': 13661, ',': 101111, 'бли': 4673, 'зи': 7148, 'вши': 4737, 'сь': 4947, 'с': 87825, 'се': 13592, 'ве': 17922, 'ра': 51775, 'о': 65117, 'сов': 6626, 'нача': 3000, 'ар': 4071, 'ти': 26576, 'й': 57483, 'ку': 14285, 'ю': 25220, 'р': 53552, 'ь': 20584, 'бу': 7244, 'ре': 30676, 'сть': 2982, 'в': 113558, 'ско': 8860, 'ни': 42740, 'ют': 8789, 'уча': 2918, 'сти': 12517, 'е': 71826, 'т': 53193, 'ж': 30621, 'ел': 13096, 'ы': 28687, 'ка': 29161, 'ли': 52

In [18]:
sorted_tokens_tuple = sorted(tokens_frequencies.items(), key=lambda item: (measure_token_length(item[0]), item[1]), reverse=True)
sorted_tokens = [token for (token, freq) in sorted_tokens_tuple]

In [19]:
print(sorted_tokens)

['представи', 'правитель', 'сообщает', '"новости', 'новости', 'россии', 'которы', 'россий', 'сообщи', 'минист', 'сообща', 'ского', 'ности', 'ствен', 'после', 'прези', 'также', 'интер', 'ность', 'будет', 'заяви', 'компа', 'милли', 'слова', 'время', 'ления', 'телей', 'челов', 'росси', 'стави', 'сообщ', 'прави', '"ново', 'ного', 'тель', 'ение', 'ения', 'стра', 'пред', 'пере', 'ской', 'ства', 'ется', 'рова', 'ство', 'ских', 'ении', 'моск', 'став', 'рабо', 'ению', 'цент', 'стви', 'года', 'прав', 'ться', 'ский', 'раль', 'стоя', 'гово', 'каза', 'лись', 'ходи', 'кото', 'ника', 'суда', 'феде', 'орга', 'стре', 'пров', 'этом', 'теля', 'полу', 'поли', 'лько', 'нача', 'дова', 'лось', 'дент', 'одна', 'ству', 'мини', 'ново', 'общи', 'про', 'при', 'что', 'ных', 'ста', 'ной', 'ова', 'пре', 'сти', 'ции', 'нов', 'под', 'сто', 'раз', 'ные', 'дел', 'ров', 'вер', 'кон', 'ско', 'его', 'ков', 'ден', 'мен', 'ово', 'ный', 'это', 'гра', 'как', 'лен', 'дол', 'ния', 'ным', 'гла', 'сов', 'ски', 'пер', 'ска', 'сво',

In [20]:
yandex_news = '''
Власти Москвы открывают запись на вакцинацию от коронавируса для жителей старше 60 лет с понедельника, 28 декабря. Об этом в своём блоге сообщил мэр столицы Сергей Собянин. Он напомнил, что Минздрав одобрил возможность прививаться от COVID-19 вакциной «Спутник V» гражданам старшего поколения.
'''

In [21]:
print(tokenize_text(string=yandex_news.lower(), sorted_tokens=sorted_tokens, unknown_token='</u>'))

['вла', 'сти', 'моск', 'вы', 'от', 'к', 'ры', 'ва', 'ют', 'за', 'пи', 'сь', 'на', 'ва', 'к', 'ци', 'на', 'ци', 'ю', 'от', 'ко', 'р', 'она', 'ви', 'ру', 'са', 'для', 'жи', 'телей', 'ста', 'р', 'ше', '6', '0', 'лет', 'с', 'по', 'не', 'дел', 'ь', 'ника', ',', '2', '8', 'де', 'ка', 'б', 'ря', 'об', 'этом', 'в', 'сво', 'ё', 'м', 'б', 'ло', 'ге', 'сообщи', 'л', 'м', 'э', 'р', 'сто', 'ли', 'ц', 'ы', 'сер', 'г', 'ей', 'соб', 'я', 'ни', 'н', 'он', 'на', 'по', 'м', 'ни', 'л,', 'что', 'ми', 'н', 'з', 'д', 'ра', 'в', 'од', 'об', 'ри', 'л', 'воз', 'мож', 'ность', 'при', 'ви', 'ва', 'ться', 'от', 'c', 'o', 'v', 'i', 'd', '-', '19', 'ва', 'к', 'ци', 'ной', '«', 'с', 'пу', 'т', 'ник', 'v', '»', 'гра', 'ж', 'дан', 'а', 'м', 'ста', 'р', 'ш', 'его', 'по', 'ко', 'ления']


Данный вариант нравится больше так как есть и больше осмысленности по токенам и больше осмысленности при токенизации.

# Ex. 2

In [23]:
df = pd.read_csv('dataset_ok.csv')

In [24]:
df.head(2)

,text,label
0,"наебалово века, для долбаёбов\n",INSULT
1,вся дума в таком же положении😁\n,NORMAL


In [25]:
df['text'].to_csv('ok_data.csv', index=False)

In [26]:
tok_sub = CharBPETokenizer()
tok_sub.train('ok_data.csv', vocab_size=2000, min_frequency=10,)

In [27]:
tok_sub.save('2k')
tok_sub = Tokenizer.from_file("2k")

In [28]:
docs = df['text'].tolist()

In [29]:
N = len(docs)
K = len(tok_sub.get_vocab())

X = lil_matrix((N, K))
X_idf = lil_matrix((N, K))

In [30]:
for i, doc in enumerate(docs):
    token_ids = tok_sub.encode(doc).ids
    for t_i in token_ids:
        X[i, t_i] += 1
        if X_idf[i, t_i] == 0:
            X_idf[i, t_i] = 1

In [31]:
idf = pd.Series(X_idf.sum(axis=0).tolist()[0])
idf = idf.apply(lambda x: np.log((1 + idf.shape[0]) / (1 + x)) + 1)

In [32]:
idf[-3:]

1997    2.226378
1998    2.202807
1999    2.241828
dtype: float64

In [33]:
X = X.multiply(lil_matrix(idf.tolist()))

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, df['label'], test_size=0.2, stratify=df['label'])

In [35]:
clf_logreg = LogisticRegression(penalty='l2', n_jobs=-1, verbose=1, max_iter=100)

In [36]:
kfold = StratifiedKFold(n_splits=7)

In [37]:
cross_val_score(clf_logreg, X, df['label'], cv=kfold, scoring='f1_macro')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    5.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    4.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    4.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    5.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    4.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    4.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1

array([0.63357952, 0.62255081, 0.63060328, 0.65005681, 0.62772651,
       0.63453144, 0.6645358 ])

In [38]:
cross_val_score(clf_logreg, X, df['label'], cv=kfold, scoring='f1_micro')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    4.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    5.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    4.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    4.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    4.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    4.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1

array([0.91054065, 0.91170751, 0.91180475, 0.91462466, 0.91229094,
       0.91122131, 0.91558884])

In [39]:
clf_logreg.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    4.2s finished


LogisticRegression(n_jobs=-1, verbose=1)

In [40]:
y_pred = clf_logreg.predict(X_test)

In [41]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      INSULT       0.81      0.58      0.68      1717
      NORMAL       0.93      0.98      0.95     12212
   OBSCENITY       0.51      0.21      0.30       136
      THREAT       0.73      0.51      0.60       333

    accuracy                           0.91     14398
   macro avg       0.74      0.57      0.63     14398
weighted avg       0.91      0.91      0.91     14398



в целом результат ок, сильно расходятся f1_micro и f1_macro, но это ввиду дисбаланса классов